In [ ]:
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

In [ ]:
df_tr = pd.read_csv('/content/drive/MyDrive/kannada_sentiment_full_train.tsv', sep='\t')
df_tt = pd.read_csv('/content/drive/MyDrive/kannada_sentiment_full_dev.tsv', sep='\t')

In [ ]:
def demoji(text):
	emoji_pattern = re.compile("["
		u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U00010000-\U0010ffff"
	"]+", flags=re.UNICODE)
	return(emoji_pattern.sub(r'', text))

In [ ]:
import re 

df_tr['text'] = df_tr['text'].apply(lambda x: demoji(x))
df_tr['text'] = df_tr['text'].apply(lambda x: x.lower())
df_tr['text'] = df_tr['text'].str.replace('\d+', '')

df_tt['text'] = df_tt['text'].apply(lambda x: demoji(x))
df_tt['text'] = df_tt['text'].apply(lambda x: x.lower())
df_tt['text'] = df_tt['text'].str.replace('\d+', '')

In [ ]:
max_len = 64

In [ ]:
import numpy as np

train_Xids = np.zeros((len(df_tr), max_len))
train_Xmask = np.zeros((len(df_tr), max_len))

val_Xids = np.zeros((len(df_tt), max_len))
val_Xmask = np.zeros((len(df_tt), max_len))

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.5MB 26.6MB/s 
     |████████████████████████████████| 901kB 42.7MB/s 
     |████████████████████████████████| 3.3MB 45.3MB/s 


In [ ]:
from transformers import BertTokenizer

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
for i, sequence in enumerate(df_tr['text']):
  train_tokens = tokenizer.encode_plus(sequence, max_length=max_len, truncation=True, padding='max_length', add_special_tokens=True, return_token_type_ids=False, return_attention_mask=True, return_tensors='tf')
  train_Xids[i, :], train_Xmask[i, :] = train_tokens['input_ids'], train_tokens['attention_mask']

for i, sequence in enumerate(df_tt['text']):
  val_tokens = tokenizer.encode_plus(sequence, max_length=max_len, truncation=True, padding='max_length', add_special_tokens=True, return_token_type_ids=False, return_attention_mask=True, return_tensors='tf')
  val_Xids[i, :], val_Xmask[i, :] = val_tokens['input_ids'], val_tokens['attention_mask']

In [ ]:
print(train_Xids)
print('=====================================================================')
print(val_Xids)

[[  101.   100.   100. ...     0.     0.     0.]
 [  101.   100.   100. ...     0.     0.     0.]
 [  101.  3565. 18906. ...     0.     0.     0.]
 ...
 [  101. 22953.  1012. ...  2678.  7293.   102.]
 [  101.   100.   100. ...     0.     0.     0.]
 [  101.  1049.  5328. ...     0.     0.     0.]]
[[  101. 14187. 22890. ...     0.     0.     0.]
 [  101. 18371.  2299. ...     0.     0.     0.]
 [  101.  2026. 10768. ...     0.     0.     0.]
 ...
 [  101.  9874.  2075. ...     0.     0.     0.]
 [  101.  1030. 18792. ...     0.     0.     0.]
 [  101.  1030.  2074. ...     0.     0.     0.]]


In [ ]:
print(train_Xmask)
print('=====================================================================')
print(val_Xmask)

[[1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]]
[[1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 ...
 [1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]]


In [ ]:
train_labels = np.asarray(pd.get_dummies(df_tr['category']))
train_labels

array([[0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0],
       ...,
       [0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0]], dtype=uint8)

In [ ]:
val_labels = np.asarray(pd.get_dummies(df_tt['category']))
val_labels

array([[1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0],
       ...,
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0]], dtype=uint8)

In [ ]:
# put all of the above arrays in tensorflow dataset object because it's faster and simpler to code

In [ ]:
import tensorflow as tf

tf.config.experimental.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
# convert data into tuples for tf
train_dataset = tf.data.Dataset.from_tensor_slices((train_Xids, train_Xmask, train_labels))
val_dataset = tf.data.Dataset.from_tensor_slices((val_Xids, val_Xmask, val_labels))

In [ ]:
for i in train_dataset.take(1):
  print(i)

(<tf.Tensor: shape=(64,), dtype=float64, numpy=
array([ 101.,  100.,  100.,  100.,  100.,  100.,  100.,  100., 1012.,
        100.,  100.,  100.,  100.,  100.,  100., 1012.,  100.,  100.,
        100.,  100.,  100.,  100.,  100.,  102.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.])>, <tf.Tensor: shape=(64,), dtype=float64, numpy=
array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])>, <tf.Tensor: shape=(5,), dtype=uint8, numpy=array([0, 1, 0, 0, 0], dtype=uint8)>)


In [ ]:
# tf dataset expectations -> input at 0th index, labels at 1st index
def mapf(input_ids, masks, labels):
  return {'input_ids': input_ids, 'attention_mask': masks}, labels

In [ ]:
train_dataset = train_dataset.map(mapf)
val_dataset = val_dataset.map(mapf)

In [ ]:
for i in train_dataset.take(1):
  print(i)

({'input_ids': <tf.Tensor: shape=(64,), dtype=float64, numpy=
array([ 101.,  100.,  100.,  100.,  100.,  100.,  100.,  100., 1012.,
        100.,  100.,  100.,  100.,  100.,  100., 1012.,  100.,  100.,
        100.,  100.,  100.,  100.,  100.,  102.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.])>, 'attention_mask': <tf.Tensor: shape=(64,), dtype=float64, numpy=
array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])>}, <tf.Tensor: shape=(5,), dtype=uint8, numpy=array([0, 1, 0, 0, 0], dtype=uint8)

In [ ]:
for i in val_dataset.take(1):
  print(i)

({'input_ids': <tf.Tensor: shape=(64,), dtype=float64, numpy=
array([  101., 14187., 22890., 18906.,  3148.,  5856.,  1057., 23806.,
       27511.,  7895.,  2204.,  2175.,  3805.,  2057.,  2024.,  9107.,
        2092.,   102.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.])>, 'attention_mask': <tf.Tensor: shape=(64,), dtype=float64, numpy=
array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])>}, <tf.Tensor: sh

In [ ]:
val_dataset.take(1)

<TakeDataset shapes: ({input_ids: (64,), attention_mask: (64,)}, (5,)), types: ({input_ids: tf.float64, attention_mask: tf.float64}, tf.uint8)>

In [ ]:
batch_size = 64
train_dataset = train_dataset.batch(batch_size, drop_remainder=True)
val_dataset = val_dataset.batch(batch_size, drop_remainder=True)

In [ ]:
val_dataset.take(1)

<TakeDataset shapes: ({input_ids: (64, 64), attention_mask: (64, 64)}, (64, 5)), types: ({input_ids: tf.float64, attention_mask: tf.float64}, tf.uint8)>

In [ ]:
from transformers import TFAutoModel

In [ ]:
bert = TFAutoModel.from_pretrained('bert-base-uncased')

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
#@title Default title text
input_ids = tf.keras.layers.Input(shape=(max_len, ), name='input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(max_len, ), name='attention_mask', dtype='int32')
embeddings = bert.bert(input_ids, attention_mask=mask)[0]
# embeddings = bert(input_ids, attention_mask=mask)[0]
# X = tf.keras.layers.GlobalMaxPool1D()(embeddings)
X = tf.keras.layers.MaxPooling1D()(embeddings)
X = tf.keras.layers.BatchNormalization()(X)
X = tf.keras.layers.Dense(1024, activation='relu')(embeddings)
X = tf.keras.layers.Dropout(0.2)(X)
X = tf.keras.layers.Dense(256, activation='relu')(X)
X = tf.keras.layers.Dropout(0.2)(X)
X = tf.keras.layers.Dense(32, activation='relu')(X)
X = tf.keras.layers.Flatten()(X)
y = tf.keras.layers.Dense(5, activation='softmax', name='outputs')(X)

model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
model.layers[2].trainable = True
model.summary()

Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 64)]         0                                            
__________________________________________________________________________________________________
attention_mask (InputLayer)     [(None, 64)]         0                                            
__________________________________________________________________________________________________
bert (TFBertMainLayer)          TFBaseModelOutputWit 109482240   input_ids[0][0]                  
                                                                 attention_mask[0][0]             
__________________________________________________________________________________________________
dense_27 (Dense)                (None, 64, 1024)     787456      bert[0][0]                 

In [ ]:
optimizer = tf.keras.optimizers.Adam(1e-4)
loss = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[acc])

In [ ]:
# cb = tf.keras.callbacks.ModelCheckpoint(save_best_only=True, filepath='/checkpoint')
cb = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', min_delta=0.001, patience=2, restore_best_weights=True)
history = model.fit(train_dataset, validation_data=val_dataset, epochs=15, callbacks=[cb])

Epoch 1/15


97/97 [==============================] - 84s 784ms/step - loss: 0.6627 - accuracy: 0.7732 - val_loss: 1.1972 - val_accuracy: 0.6406
Epoch 2/15
97/97 [==============================] - 80s 822ms/step - loss: 0.5367 - accuracy: 0.8170 - val_loss: 1.2372 - val_accuracy: 0.6641
Epoch 3/15
97/97 [==============================] - 79s 812ms/step - loss: 0.4506 - accuracy: 0.8454 - val_loss: 1.4619 - val_accuracy: 0.6562
Epoch 4/15
97/97 [==============================] - 80s 830ms/step - loss: 0.4101 - accuracy: 0.8628 - val_loss: 1.3979 - val_accuracy: 0.6797
Epoch 5/15
97/97 [==============================] - 83s 861ms/step - loss: 0.3562 - accuracy: 0.8808 - val_loss: 1.5070 - val_accuracy: 0.6844
Epoch 6/15
97/97 [==============================] - 77s 791ms/step - loss: 0.3514 - accuracy: 0.8850 - val_loss: 1.5694 - val_accuracy: 0.6469
Epoch 7/15
97/97 [==============================] - 79s 815ms/step - loss: 0.3092 - accuracy: 0.8980 - val_loss: 1.8890 - val_accuracy: 0.6422


In [ ]:
l, a = model.evaluate(val_dataset)

10/10 [==============================] - 3s 250ms/step - loss: 1.5070 - accuracy: 0.6844


In [ ]:
y_pred = model.predict(x = [val_Xids, val_Xmask])
y_pred

array([[9.95220721e-01, 2.61009554e-03, 1.56209036e-03, 2.43579552e-06,
        6.04660599e-04],
       [1.29944761e-03, 4.78567352e-04, 9.96696115e-01, 7.39922223e-04,
        7.85975484e-04],
       [1.10301943e-02, 6.54273806e-03, 9.57602799e-01, 1.79387331e-02,
        6.88559562e-03],
       ...,
       [2.39977799e-03, 2.21281359e-03, 5.41519582e-01, 4.41598654e-01,
        1.22691439e-02],
       [1.83817476e-03, 9.98089612e-01, 5.99841696e-05, 1.06930834e-07,
        1.20797686e-05],
       [9.79068037e-03, 9.89944220e-01, 2.48778408e-04, 4.89729359e-07,
        1.59343872e-05]], dtype=float32)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
a = accuracy_score(np.argmax(val_labels, axis=1), np.argmax(y_pred, axis=1))
a

0.6743849493487699

In [ ]:
f = f1_score(np.argmax(val_labels, axis=1), np.argmax(y_pred, axis=1), average='weighted')
f

0.6588934992383051

In [ ]:
len(y_pred)

691

In [ ]:
len(val_labels)

691